# Import dependencies and instantiate model and import

In [1]:
import torch
import noisynn as nnn 
import timm 

model = timm.create_model('convnext_base.fb_in22k_ft_in1k', pretrained=False)
inp = torch.rand(4,3,224,224)

To check the name of the layer you want to add NoisyNN to, use model.named_modules() to get the name of the layer you want to add NoisyNN to.

In this tutorial, I choose the layers to be `layers = ['stages.3.blocks.2','stages.2.blocks.26']` of the ConvNeXt-B model from `timm`.

In [2]:
# print(dict(model.named_modules()).keys())

layers = ['stages.3.blocks.2','stages.2.blocks.26']

# Manage NoisyNN 

## Manually

### Instantiate NoisyNN instances to the model

`n_layers_inject_per_batch`: max number of layers activated NoisyNN in a batch. The real number of layers activated will be `max(len(layers), n_layers_inject_per_batch)`

`add_noise_fn`: a `Callale` that do transformation on the layer(s) output specified above. If `None`, default add_noise function will be used. More in `default_function.py`

`debug_fn`: a `Callale` that is called when in debug mode. If `None`, default debug function will be used. More in `default_function.py` 

In [3]:
nnn.inject_noisy_nn(model=model, layers_name= layers, n_layers_inject_per_batch= 1, add_noise_fn= None, debug_fn= None, inplace= True, verbose= True);
# model_new = nnn.inject_noisy_nn(model=model, layers_name= layers, n_layers_inject_per_batch= 1, inplace= False, verbose= True)   <-- In case you don't want to mess with the original model

<--Trying to inject NoisyNN instances!-->
---NoisyNN instance injected onto layer stages.2.blocks.26.---
---NoisyNN instance injected onto layer stages.3.blocks.2.---
<--All chosen layers are injected with NoisyNN instance!-->


### Doing what ever you want here with NoisyNN injected model

In [4]:
with nnn.debug_mode(True):   # <-- Just call the debug function, in this case will print the chosen layer.
    model(inp);

Layer stages.2.blocks.26 is chosen!


### Remove NoisyNN instances from the model

In [5]:
nnn.remove_noisy_nn(model=model, inplace= True, verbose= True);
# model_new = nnn.remove_noisy_nn(model=model, inplace= False, verbose= True)  <-- In case you want to keep the NoisyNN injected model?

<---Trying to remove NoisyNN!!!--->
---NoisyNN instance removed from layer stages.2.blocks.26.---
---NoisyNN instance removed from layer stages.3.blocks.2.---
<--All injected NoisyNN instances have been removed!-->


## Automatically

### As a context manager

In [6]:
with nnn.noisy_nn(model, layers_name= layers, add_noise_fn= None, debug_fn= None):
    with nnn.debug_mode(enabled= True): # <-- Just call the debug function, in this case will print the chosen layer.
        model(inp);

<--Trying to inject NoisyNN instances!-->
---NoisyNN instance injected onto layer stages.2.blocks.26.---
---NoisyNN instance injected onto layer stages.3.blocks.2.---
<--All chosen layers are injected with NoisyNN instance!-->
Layer stages.3.blocks.2 is chosen!
<---Trying to remove NoisyNN!!!--->
---NoisyNN instance removed from layer stages.2.blocks.26.---
---NoisyNN instance removed from layer stages.3.blocks.2.---
<--All injected NoisyNN instances have been removed!-->


### As a decorator

In [7]:
@nnn.noisy_nn(model, layers_name= layers, add_noise_fn= None, debug_fn= None)
def infer():
    model(inp);

with nnn.debug_mode(enabled= True): # <-- Just call the debug function, in this case will print the chosen layer.
    infer()

<--Trying to inject NoisyNN instances!-->
---NoisyNN instance injected onto layer stages.2.blocks.26.---
---NoisyNN instance injected onto layer stages.3.blocks.2.---
<--All chosen layers are injected with NoisyNN instance!-->
Layer stages.3.blocks.2 is chosen!
<---Trying to remove NoisyNN!!!--->
---NoisyNN instance removed from layer stages.2.blocks.26.---
---NoisyNN instance removed from layer stages.3.blocks.2.---
<--All injected NoisyNN instances have been removed!-->


# Debugging NoisyNN 


## Debug mode
Currently, default debugging only support printing out the chosen layer name. For further customization, see `default_function.py`

### Global mode

In [8]:
# Entering debug mode by using
nnn.enable_debug_mode()
#or
# nnn.debug_mode(enabled= True) #<-- This support for deciding fallback debug mode

#Your code execution with debug here
infer()

# Disable debug mode by using
nnn.disable_debug_mode()

infer() # <-- No debug here

<--Trying to inject NoisyNN instances!-->
---NoisyNN instance injected onto layer stages.2.blocks.26.---
---NoisyNN instance injected onto layer stages.3.blocks.2.---
<--All chosen layers are injected with NoisyNN instance!-->
Layer stages.2.blocks.26 is chosen!
<---Trying to remove NoisyNN!!!--->
---NoisyNN instance removed from layer stages.2.blocks.26.---
---NoisyNN instance removed from layer stages.3.blocks.2.---
<--All injected NoisyNN instances have been removed!-->
<--Trying to inject NoisyNN instances!-->
---NoisyNN instance injected onto layer stages.2.blocks.26.---
---NoisyNN instance injected onto layer stages.3.blocks.2.---
<--All chosen layers are injected with NoisyNN instance!-->
<---Trying to remove NoisyNN!!!--->
---NoisyNN instance removed from layer stages.2.blocks.26.---
---NoisyNN instance removed from layer stages.3.blocks.2.---
<--All injected NoisyNN instances have been removed!-->


### As a context manager

See above section for example usage

### As a decorator

In [9]:
@nnn.debug_mode(enabled= True)
@nnn.noisy_nn(model, layers_name= layers, add_noise_fn= None, debug_fn= None)
def infer():
    model(inp);
infer()

<--Trying to inject NoisyNN instances!-->
---NoisyNN instance injected onto layer stages.2.blocks.26.---
---NoisyNN instance injected onto layer stages.3.blocks.2.---
<--All chosen layers are injected with NoisyNN instance!-->
Layer stages.3.blocks.2 is chosen!
<---Trying to remove NoisyNN!!!--->
---NoisyNN instance removed from layer stages.2.blocks.26.---
---NoisyNN instance removed from layer stages.3.blocks.2.---
<--All injected NoisyNN instances have been removed!-->


## Safety checker

Current implementation have three `level`s of safety checker: 
- `0`  &emsp;&emsp;&emsp;&ensp;&ensp; : No safety checker at all.
- `1`   (default) &ensp;: Only warnings if there is a mismatch (type, shape, etc.) between output before and after adding noise, excluding the data.
- `2`  &emsp;&emsp;&emsp;&ensp;&ensp; : Raise error if there is a mismatch (type, shape, etc.) between output before and after adding noise, but not the content/data.

The syntax should be identical to debug_mode, please see the above examples for usage.